In [1]:
import getEPH
import functionsForModels
import make_dummy
import schoolYears
import categorize
import functionsForModels
import createVariables

import pandas as pd
#http://statsmodels.sourceforge.net/devel/examples/generated/example_wls.html
import numpy as np
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.iolib.table import (SimpleTable, default_txt_fmt)
np.random.seed(1024)
%matplotlib inline

/home/pipe/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
getEPH.getEPHdbf('t310')

('Downloading', 't310')
file in place, creating CSV file
csv file cleanDataHousehold t310 .csv successfully created in folder data/
csv file cleanData t310 .csv successfully created in folder data/


In [ ]:
data1 = pd.read_csv('data/cleanDatat310.csv')
data1.columns

In [ ]:
data2 = categorize.categorize(data1)

In [ ]:
data3 = schoolYears.schoolYears(data2)
data4 = createVariables.createVariables(data3)

In [ ]:
data4.columns

# modelo para predecir ingreso individual

In [ ]:
#considerar los ingresos del decil 0 como los trabajamos
jobsAndIncome = (data4.activity == 1) & (data4.P21 > 1) 
headAndSpouse = (data4.familyRelation == 1)|(data4.familyRelation == 2)
dataParaModelo = data4.copy().loc[jobsAndIncome,:]


In [ ]:
variablesOfInterest = ['age',
                              'age2',
                              'female',
                              'education',
                              'education2']

In [ ]:
model = functionsForModels.runModel(dataset = dataParaModelo, income = 'lnIncome', variables = variablesOfInterest)

In [ ]:
X = sm.add_constant(dataParaModelo.copy().loc[:,variablesOfInterest].values)

In [ ]:
dataParaModelo['predictedLnIncome'] = model.predict(X)

In [ ]:
#dataParaModelo.loc[:,['id','predictedLnIncome']]

# merge con la base individual

In [ ]:
dataParaModelo.columns

In [ ]:
data4.activity.value_counts()

In [ ]:
data4.columns

In [ ]:
data4.shape

In [ ]:
paraMerge = dataParaModelo.loc[:,['CODUSU', 'NRO_HOGAR', 'COMPONENTE','predictedLnIncome']]
print paraMerge.shape


In [ ]:
paraMerge.head()

In [ ]:
data4.shape

In [ ]:
data = pd.merge(left = data4 , right = paraMerge, on = ['CODUSU', 'NRO_HOGAR', 'COMPONENTE'], how = 'left')

In [ ]:
data.shape

In [ ]:
data.predictedLnIncome[data.predictedLnIncome.isnull()] = 0

In [ ]:
data.head()

In [ ]:
data['job'] = (data.activity==1).astype(int)

In [ ]:
data.columns

In [ ]:
cleanData = data.copy().loc[(headAndSpouse),
                              ['id',
                            'familyRelation',
                              'age',
                              'age2',
                              'female',
                              'education',
                              'education2',
                              'primary',
                              'secondary',
                              'university',
                              'P21',
                              'P47T',
                              'lnIncome',
                               u'lnIncomeT',
                              'predictedLnIncome',
                              'job',
                              'DECCFR',
                               'DECIFR',
                               'maritalStatus',
                               'reading',
                               'placeOfBirth'
                              ]]

In [ ]:
cleanData.head()

In [ ]:
pivot = cleanData.pivot(index='id', columns='familyRelation')
pivot.head()

In [ ]:
pivot.columns

In [ ]:
pivot.to_csv('data/pivotInd.csv')

In [ ]:
#para leer en otro archivo
dataN = pd.read_csv('data/pivotInd.csv', names = ['id','headAge','spouseAge','headAge2','spouseAge2',
                                                  'headFemale','spouseFemale','headEduc','spouseEduc',
                                                  'headEduc2','spouseEduc2','headPrimary','spousePrimary',
                                                  'headSecondary','spouseSecondary','headUniversity','spouseUniversity',
                                                  'headP21','spouseP21','headP47T','spouseP47T',
                                                  'headLnIncome','spouseLnIncome','headLnIncomeT','spouseLnIncomeT',
                                                  'headPredictedLnIncome','spousePredictedLnIncome','headJob','spouseJob',
                                                  'headDECCFR','spouseDECCFR','headDECIFR','spouseDECIFR',
                                                  'headMaritalStatus','spouseMaritalStatus',
                                                  'headReading','spouseReading','headPlaceOfBirth','spouseplaceOfBirth',
                                                 ],skiprows = 3)
dataN.head()

In [ ]:
np.where(np.isnan(pivot.values),0,pivot.values)

dataN = dataN.dropna(axis = 0)
dataN.head()


In [ ]:
pivot.shape